## 서울 열린 데이터 광장 홈페이지에서 공공데이터 목록들을 수집해보자

In [1]:
import requests as req
import pandas as pd
from bs4 import BeautifulSoup as bs
from tqdm import tqdm

In [2]:
U_A = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36"}

In [3]:
res_seoul = req.get("https://data.seoul.go.kr/dataList/datasetList.do", headers=U_A)
res_seoul

<Response [200]>

In [4]:
soup = bs(res_seoul.text, "lxml")

# lxml : C언어로 작성된 파서로 큰 문서를 다룰 때 속도가 빨라서 유리하며 HTML구조를 잘 보정하여 피싱해줌, XML에 대한 피싱도 지원함.
# html.parser : 파이썬 표준 라이브러리에 내장된 HTML파서로 상대적으로 느리며 보정력이 좋지 못함, XML 파싱 미지원

### 데이터 목록 정보 수집
- 타이틀
- 분야
- 수정일자
- 제공부서

In [5]:
# 1. 타이틀
title = soup.select("a.goView > strong")
title

[<strong>서울글로벌센터 접수유형 및 상담유형별(월) 상담실적 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>상상대로 서울 자유제안 정보 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 하수관로 수위 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울글로벌센터 상담유형별(월) 상담실적 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울글로벌센터 접수유형별 언어별 상담실적 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울글로벌센터 상담유형별(분기) 상담실적 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 지역 시간별 수질 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 위생업소 전체 행정처분내역 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 실시간 자치구별 대기환경 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 도로변/입체대기 측정소별 실시간 대기환경 현황

In [17]:
# 2. 분야
soup.select("dl.type-b em:nth-child(2)")                # 자손은 그냥 띄어쓰기 후 두번째 태그를 원하는 것이니 nth를 씀.

[<em class="stat-class-1">[보건]</em>,
 <em class="stat-class-1">[보건]</em>,
 <em class="stat-class-1">[보건]</em>,
 <em class="stat-class-1">[보건]</em>,
 <em class="stat-class-5">[복지]</em>,
 <em class="stat-class-2">[일반행정]</em>,
 <em class="stat-class-8">[도시관리]</em>,
 <em class="stat-class-5">[복지]</em>,
 <em class="stat-class-1">[보건]</em>,
 <em class="stat-class-5">[복지]</em>]

In [42]:
category = soup.select("dl.type-b em:nth-child(2)")      
category

[<em class="stat-class-5">[복지]</em>,
 <em class="stat-class-8">[도시관리]</em>,
 <em class="stat-class-6">[환경]</em>,
 <em class="stat-class-6">[환경]</em>,
 <em class="stat-class-12">[주택/건설]</em>,
 <em class="stat-class-12">[주택/건설]</em>,
 <em class="stat-class-6">[환경]</em>,
 <em class="stat-class-6">[환경]</em>,
 <em class="stat-class-8">[도시관리]</em>,
 <em class="stat-class-6">[환경]</em>]

In [24]:
# 3. 수정일자
date = soup.select("dd.list-statistics-info2 span:nth-child(1)")

for i in date :
    # 년월일은 자릿수가 정해져 있으므로 텍스트 슬라이싱 적용!
    print(i.text.strip()[-10:])                  # 뒤에서 10번째부터 이므로

2025-07-10
2025-07-10
2025-07-10
2025-07-10
2025-07-10
2025-07-10
2025-07-10
2025-07-10
2025-07-10
2025-07-10


In [33]:
# 4. 제공부서
department = soup.select("dd.list-statistics-info2 span:nth-child(3)")
for i in department :
    # 수정일자와는 다르게 자릿수가 정해져 있지 않지만 앞의 텍스트와 공백의 길이가 일정하므로 그 길이를 파악하여 인덱스를 추정!
    print(i.text.strip()[19:])

디지털도시국 정보시스템과
시민건강국 감염병관리과
디지털도시국 데이터전략과
디지털도시국 데이터전략과
경제실 경제일자리기획관 금융투자과
홍보기획관 콘텐츠담당관
물순환안전국 물재생계획과
경제실 경제일자리기획관 금융투자과
디지털도시국 데이터전략과
경제실 경제일자리기획관 금융투자과


In [34]:
len("제공부서 :  										")

18

### GET, POST 통신 방식
두 방식 모두 HTTP 내에서 클라이언트가 서버에 데이터를 요청하거나 전송할 때 사용하는 방식
- **GET** : 데이터가 URL에 노출됨(데이터의 길이 제한이 있으며 일반적으로 2048자 까지 제한)
- **POST** : GET보다 보안이 높은 방식으로 데이터가 URL에 노출되지 않음(전송 데이터의 종류 및 크기 제한이 없음)

In [38]:
url = "https://data.seoul.go.kr/dataList/datasetList.do"
payload = {"pageIndex":2}        # 2패이지를 접속하고 싶다./페이지는 사이트마다 다름 확인해봐야함.

# post : post방식으로 요청 보내
 # data 인자는 폼 데이터를 설정하는 매개변수로 딕셔너리로 설정가능하며 웹 접근시 제출해야하는 정보를 포함함
res_seoul = req.post(url, data=payload, headers=U_A)
res_seoul

<Response [200]>

In [39]:
soup = bs(res_seoul.text, "lxml")

In [40]:
title = soup.select("a.goView > strong")
title

[<strong>서울글로벌센터 접수유형별 언어별 상담실적 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 지역 시간별 수질 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 도로변/입체대기 측정소별 실시간 대기환경 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 실시간 자치구별 대기환경 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 집수리닷컴 공지사항 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 집수리 시공업체 정보 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 기간별 시간평균 대기환경 정보 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 시간 평균 대기오염도 정보 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>집수리 아카데미 수료자 활동내역 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 권역별 오존 경보 현황 <img alt="업데이트" src=

### 1~5page의 정보(4종류)를 수집해서 DF로 나타내보세요. 

In [ ]:
url = "https://data.seoul.go.kr/dataList/datasetList.do"

titles_list, categories_list, dates_list, departments_list = [],[],[],[]

# 1~5 page 접근
for i in tqdm(range(1, 6)) :
    payload = {"pageIndex": i}   # 페이지 숫자
    res_seoul = req.post(url, data=payload, headers=U_A)
    soup = bs(res_seoul.text, "lxml")

    title = soup.select("a.goView > strong")
    category = soup.select("div.list-statistics em:nth-child(2)")
    date = soup.select("dd.list-statistics-info2 span:nth-child(1)")
    department = soup.select("dd.list-statistics-info2 span:nth-child(3)")

    # 페이지별 수집할 정보들 리스트에 넣어주기(10회 반복)
    for j in range(len(title)) :
        titles_list.append(title[j].text)
        categories_list.append(category[j].text)
        dates_list.append(date[j].text.strip()[-10:])
        departments_list.append(department[j].text.strip()[19:])

seoul_dic = {"타이틀":titles_list, "분야":categories_list, 
             "수정일자":dates_list, "제공부서":departments_list}

seoul_df = pd.DataFrame(seoul_dic, index=range(1,51))
seoul_df.index.name = "No."
seoul_df